# colab용 자동화

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pickle
# from soynlp.hangle import levenshtein
# # from PreProcessing.find_common_part
# from konlpy.tag import *
# from PreProcessing import find_common_part
import numpy as np
from gensim.models import Word2Vec
from keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# 불러올때    
with open('/content/drive/Shared drives/BigData/team__KU/data/result.txt', 'rb') as f:
    result = pickle.load(f)
    
result_gensim_input = [_.split() for _ in result if _ != '']
result_tokenizer_input = [v for i, v in enumerate(result) if i%2 == 0 and v != '']
ebs_in_result_for_getting_max_len = [v for i, v in enumerate(result_gensim_input) if i%2 == 0 and v != []]
ebs_in_result_gensim_input = [v for i, v in enumerate(result_gensim_input) if i%2 == 0 and v != [] ]
google_in_result_gensim_input = [v for i, v in enumerate(result_gensim_input) if i%2 == 1 and v != []]

# Generate EBS string vectors matrix
ws = 1
es = 30

model_cbow = Word2Vec(
                ebs_in_result_gensim_input, 
                window = ws,
                size =es,
                min_count=1,
                workers = 10
                )

vocabs = list(model_cbow.wv.index2word)
embedding_matrix = np.zeros((len(vocabs), es))
for i, w in enumerate(vocabs):
    embedding_matrix[i] = model_cbow[w]

#embedding_matrix 맨 위에 0벡터 추가
stacked_zero = np.zeros((1, es))
embedding_matrix = np.vstack((stacked_zero, embedding_matrix))

t = Tokenizer()
t.fit_on_texts(result_tokenizer_input)
vocab_size = len(t.word_index) + 1

#시퀀스 만들기
sequences = list()
for line in result_tokenizer_input: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[i-9 if i > 9 else 0:i+1]
        sequences.append(sequence)

y = []
for _ in np.array(sequences):
    y.append(_.pop(-1))
    
X = np.array(sequences)
y = np.array(y)
        
max_len=max(len(l) for l in X)
print('max length : {}'.format(max_len))

X = pad_sequences(X, maxlen=max_len, padding='post')

y = to_categorical(y, num_classes=vocab_size)

embedding_layer = Embedding(vocab_size,
                            es,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

model = Sequential()
model.add(embedding_layer)
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(128, activation = 'relu'))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size = 40, epochs=400, verbose=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


max length : 9
Epoch 1/400
1247/1247 - 39s - loss: 7.7912 - accuracy: 0.0162
Epoch 2/400
1247/1247 - 38s - loss: 7.4248 - accuracy: 0.0196
Epoch 3/400
1247/1247 - 37s - loss: 7.2827 - accuracy: 0.0230
Epoch 4/400
1247/1247 - 38s - loss: 7.1323 - accuracy: 0.0236
Epoch 5/400
1247/1247 - 37s - loss: 7.0053 - accuracy: 0.0247
Epoch 6/400
1247/1247 - 37s - loss: 6.8592 - accuracy: 0.0268
Epoch 7/400
1247/1247 - 37s - loss: 6.7152 - accuracy: 0.0287
Epoch 8/400
1247/1247 - 37s - loss: 6.5793 - accuracy: 0.0308
Epoch 9/400
1247/1247 - 40s - loss: 6.4318 - accuracy: 0.0309
Epoch 10/400
1247/1247 - 38s - loss: 6.2726 - accuracy: 0.0321
Epoch 11/400
1247/1247 - 37s - loss: 6.1149 - accuracy: 0.0335
Epoch 12/400
1247/1247 - 38s - loss: 5.9435 - accuracy: 0.0341
Epoch 13/400
1247/1247 - 38s - loss: 5.7574 - accuracy: 0.0380
Epoch 14/400
1247/1247 - 37s - loss: 5.5634 - accuracy: 0.0488
Epoch 15/400
1247/1247 - 37s - loss: 5.3775 - accuracy: 0.0701
Epoch 16/400
1247/1247 - 37s - loss: 5.1991 - acc